In [ ]:
import customtkinter
import xlwings as xw
import pandas as pd
import os
import warnings

# Suppress the specific UserWarning about Data Validation in openpyxl
warnings.filterwarnings("ignore", message="Data Validation extension is not supported and will be removed", category=UserWarning)

from tkinter import filedialog, messagebox, StringVar, IntVar
from difflib import get_close_matches  # To find close matches for parameter names

# Setting up customtkinter appearance and theme
customtkinter.set_appearance_mode("System")  # Modes: system (default), light, dark
customtkinter.set_default_color_theme("blue")  # Themes: blue (default), dark-blue, green

MAX_FILENAME_DISPLAY_LENGTH = 40  # Limit the length of the file display

class App:
    def __init__(self, root):
        self.root = root
        self.root.geometry("600x400")
        self.root.title("Abacus Data Sync Tool")

        # Labels for instructions (Increased Font Size)
        self.label_china = customtkinter.CTkLabel(root, text="Select the Baseline File:", font=("Arial", 16))
        self.label_china.grid(row=0, column=0, padx=10, pady=10, sticky="w")

        self.label_abacus = customtkinter.CTkLabel(root, text="Select the Abacus Template File:", font=("Arial", 16))
        self.label_abacus.grid(row=1, column=0, padx=10, pady=10, sticky="w")

        self.label_output = customtkinter.CTkLabel(root, text="Select the Output Directory:", font=("Arial", 16))
        self.label_output.grid(row=2, column=0, padx=10, pady=10, sticky="w")

        self.label_uploaded = customtkinter.CTkLabel(root, text="Select the Data File:", font=("Arial", 16))
        self.label_uploaded.grid(row=3, column=0, padx=10, pady=10, sticky="w")

        # Buttons for file selection (Increased Font Size)
        self.btn_china = customtkinter.CTkButton(root, text="Browse...", command=self.select_china_file, font=("Arial", 16))
        self.btn_china.grid(row=0, column=1, padx=10, pady=10)

        self.btn_abacus = customtkinter.CTkButton(root, text="Browse...", command=self.select_abacus_file, font=("Arial", 16))
        self.btn_abacus.grid(row=1, column=1, padx=10, pady=10)

        self.btn_output = customtkinter.CTkButton(root, text="Browse...", command=self.select_output_dir, font=("Arial", 16))
        self.btn_output.grid(row=2, column=1, padx=10, pady=10)

        self.btn_uploaded = customtkinter.CTkButton(root, text="Browse...", command=self.select_uploaded_file, font=("Arial", 16))
        self.btn_uploaded.grid(row=3, column=1, padx=10, pady=10)

        # Process button (Increased Font Size)
        self.btn_process = customtkinter.CTkButton(root, text="Start Processing", command=self.choose_process_option, font=("Arial", 16))
        self.btn_process.grid(row=4, columnspan=2, pady=20)

        # File paths
        self.china_file = None
        self.abacus_file = None
        self.output_dir = None
        self.uploaded_file = None

        # To store user selections
        self.selected_schedule_sheet = None
        self.uploaded_xl = None  # To store the uploaded Excel file
        self.parameter_columns = []
        self.selected_headers = {}
        self.schedule_df = None
        self.sheet_names_2024 = []  # Store sheet names
        self.current_sheet_index = 0  # To keep track of the current sheet being processed
        self.selected_sheets = []  # For storing selected individual sheets

    # Helper function to truncate file paths
    def truncate_filename(self, filepath):
        if len(filepath) > MAX_FILENAME_DISPLAY_LENGTH:
            return f"...{filepath[-MAX_FILENAME_DISPLAY_LENGTH:]}"
        return filepath

    # File selection methods
    def select_china_file(self):
        self.china_file = filedialog.askopenfilename(title="Select the Baseline File", filetypes=[("Excel files", "*.xlsx *.xlsm")])
        if self.china_file:
            truncated_path = self.truncate_filename(self.china_file)
            self.label_china.configure(text=f"Selected: {truncated_path}")

    def select_abacus_file(self):
        self.abacus_file = filedialog.askopenfilename(title="Select the Abacus Template File", filetypes=[("Excel files", "*.xlsm")])
        if self.abacus_file:
            truncated_path = self.truncate_filename(self.abacus_file)
            self.label_abacus.configure(text=f"Selected: {truncated_path}")

    def select_output_dir(self):
        self.output_dir = filedialog.askdirectory(title="Select the Output Directory")
        if self.output_dir:
            truncated_path = self.truncate_filename(self.output_dir)
            self.label_output.configure(text=f"Selected: {truncated_path}")

    def select_uploaded_file(self):
        self.uploaded_file = filedialog.askopenfilename(title="Select the Data File", filetypes=[("Excel files", "*.xlsx *.xlsm")])
        if self.uploaded_file:
            truncated_path = self.truncate_filename(self.uploaded_file)
            self.label_uploaded.configure(text=f"Selected: {truncated_path}")

    # Updated method to ensure the new window is always on top and in focus
    def choose_process_option(self):
        if not self.china_file or not self.abacus_file or not self.output_dir or not self.uploaded_file:
            messagebox.showerror("Error", "Please select all required files and directory.")
            return

        # New window for choosing the processing option
        option_window = customtkinter.CTkToplevel(self.root)
        option_window.title("Choose Processing Option")
        option_window.geometry("500x250")  # Adjusted window size
        option_window.lift()  # Bring window to the front
        option_window.attributes('-topmost', True)  # Keep it on top
        option_window.focus_force()  # Force focus on the new window

        customtkinter.CTkLabel(option_window, 
                               text="Choose how you want to process the Mediums from the Baseline file:", 
                               font=("Arial", 16), 
                               wraplength=480,  # Text wrapping to fit within window
                               anchor="center").pack(padx=10, pady=10)

        customtkinter.CTkButton(option_window, text="Process All Mediums", command=lambda: self.process_files(option_window, all_sheets=True), font=("Arial", 16)).pack(pady=10)
        customtkinter.CTkButton(option_window, text="Select Individual Mediums", command=lambda: self.process_files(option_window, all_sheets=False), font=("Arial", 16)).pack(pady=10)

    # Function to find the closest matching header
    def get_best_match(self, target, options):
        matches = get_close_matches(target, options, n=1, cutoff=0.5)
        if matches:
            return matches[0]
        return "None"

    # File processing and remaining methods (from your original code)
    def process_files(self, option_window, all_sheets):
        option_window.destroy()
        try:
            china_xl = pd.ExcelFile(self.china_file)
            self.sheet_names_2024 = [sheet for sheet in china_xl.sheet_names if '2024' in sheet]

            self.app = xw.App(visible=False)  # Runs Excel in the background
            self.wb = self.app.books.open(self.abacus_file)

            self.uploaded_xl = pd.ExcelFile(self.uploaded_file)

            if all_sheets:
                self.process_next_sheet()
            else:
                self.show_sheet_selection_window()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
            if hasattr(self, 'wb') and self.wb:
                self.wb.close()
            if hasattr(self, 'app') and self.app:
                self.app.quit()

    def show_sheet_selection_window(self):
        sheet_window = customtkinter.CTkToplevel(self.root)
        sheet_window.title("Select Mediums to Process")
        sheet_window.geometry("400x300")
        sheet_window.lift()  # Bring window to the front
        sheet_window.attributes('-topmost', True)  # Keep it on top
        sheet_window.focus_force()  # Force focus on the new window

        customtkinter.CTkLabel(sheet_window, text="Select the Mediums you want to process:", font=("Arial", 16)).pack(padx=10, pady=10)

        frame = customtkinter.CTkFrame(sheet_window)
        frame.pack(padx=10, pady=10)

        self.sheet_vars = {}
        for sheet_name in self.sheet_names_2024:
            var = IntVar()
            customtkinter.CTkCheckBox(frame, text=sheet_name, variable=var, font=("Arial", 16)).pack(anchor="w")
            self.sheet_vars[sheet_name] = var

        customtkinter.CTkButton(sheet_window, text="Confirm", command=lambda: self.get_selected_sheets(sheet_window), font=("Arial", 16)).pack(pady=10)

    def get_selected_sheets(self, sheet_window):
        self.selected_sheets = [sheet for sheet, var in self.sheet_vars.items() if var.get() == 1]

        if not self.selected_sheets:
            messagebox.showerror("Error", "Please select at least one Baseline Medium to process.")
            return

        sheet_window.destroy()
        self.process_selected_sheets()

    def process_selected_sheets(self):
        self.sheet_names_2024 = self.selected_sheets
        self.current_sheet_index = 0
        self.process_next_sheet()

    def process_next_sheet(self):
        if self.current_sheet_index >= len(self.sheet_names_2024):
            self.finish_processing()
            return

        sheet_name = self.sheet_names_2024[self.current_sheet_index]
        print(f"Processing sheet: {sheet_name}")

        df = pd.read_excel(self.china_file, sheet_name=sheet_name, header=2)  # Header is at the 3rd row (index 2)
        df.columns = df.columns.str.strip()

        if 'Ratecard Spend' in df.columns:
            start_col = df.columns[1]
            end_col = 'Ratecard Spend'

            selected_cols = df.loc[:, start_col:end_col]

            baseline_ws = self.wb.sheets['Baseline']
            baseline_ws.range("A3:O99999").clear_contents()  # Clear the expanded range
            baseline_ws.range(f"A3").value = selected_cols.values

            ratecard_index = df.columns.get_loc('Ratecard Spend')
            next_cols = df.iloc[:, ratecard_index + 1: ratecard_index + 3]
            baseline_ws.range("K3:L3").value = next_cols.values

            baseline_col_name = '2024 Baseline\n USE IN ABACUS'
            if baseline_col_name in df.columns:
                baseline_col_data = df[baseline_col_name].values.reshape(-1, 1)
                baseline_ws.range("M3").value = baseline_col_data

            max_row = df.shape[0]
            a_column_data = df.iloc[:max_row, 0].values.reshape(-1, 1)
            baseline_ws.range("O3").value = a_column_data

            parameters_ws = self.wb.sheets['Parameters']
            parameters_ws.range("K15:K100").clear_contents()
            valid_cols = [col for col in selected_cols.columns if col and 'Unnamed' not in col and col != 'Ratecard Spend']
            for idx, col_name in enumerate(valid_cols, start=15):
                parameters_ws.range(f"K{idx}").value = col_name

            available_sheets = ["None"] + self.uploaded_xl.sheet_names
            self.show_selection_window(available_sheets, f"Select Current Year Data Sheet for {sheet_name}", lambda: self.process_selected_schedule(sheet_name))
        else:
            print(f"'Ratecard Spend' column not found in {sheet_name}. Skipping this sheet.")
            self.current_sheet_index += 1
            self.process_next_sheet()

    def show_selection_window(self, available_options, prompt, callback):
        selection_window = customtkinter.CTkToplevel(self.root)
        selection_window.title(prompt)
        selection_window.geometry("500x250")  # Adjusted window size
        selection_window.lift()  # Bring window to the front
        selection_window.attributes('-topmost', True)  # Keep it on top
        selection_window.focus_force()  # Force focus on the new window

        selected_option = StringVar(selection_window)
        selected_option.set("None")

        dropdown = customtkinter.CTkOptionMenu(selection_window, variable=selected_option, values=available_options, font=("Arial", 16))
        dropdown.pack(padx=10, pady=10)

        def on_confirm():
            self.selected_schedule_sheet = selected_option.get()
            selection_window.destroy()
            callback()

        customtkinter.CTkButton(selection_window, text="Confirm", command=on_confirm, font=("Arial", 16)).pack(pady=10)

    def process_selected_schedule(self, sheet_name):
        print(f"User selected: {self.selected_schedule_sheet}")

        if self.selected_schedule_sheet == "None":
            print(f"Skipping schedule sheet processing for {sheet_name}.")
            self.save_workbook(sheet_name)
        else:
            if self.selected_schedule_sheet in self.uploaded_xl.sheet_names:
                schedule_df = pd.read_excel(self.uploaded_file, sheet_name=self.selected_schedule_sheet, header=None)
                brand_row_index = schedule_df[schedule_df.iloc[:, 0] == "Brand"].index
                if not brand_row_index.empty:
                    header_row_index = brand_row_index[0]
                    self.schedule_df = pd.read_excel(self.uploaded_file, sheet_name=self.selected_schedule_sheet, header=header_row_index)
                    self.full_headers = self.schedule_df.columns[1:].astype(str).str.strip().tolist()
                    self.full_headers.append("None")
                    print(f"Headers found after 'Brand': {self.full_headers}")
                    parameters_ws = self.wb.sheets['Parameters']
                    self.parameter_columns = parameters_ws.range("K15:K100").value
                    self.parameter_columns = [col for col in self.parameter_columns if col and 'Unnamed' not in col]
                    self.selected_headers = {}
                    self.show_multiple_header_selection_window(self.parameter_columns, self.full_headers, self.wb, sheet_name)
                else:
                    print(f"'Brand' column not found in column A of {self.selected_schedule_sheet}.")
                    self.current_sheet_index += 1
                    self.process_next_sheet()
            else:
                print(f"Selected schedule sheet '{self.selected_schedule_sheet}' is not in available sheets.")
                self.current_sheet_index += 1
                self.process_next_sheet()

    def show_multiple_header_selection_window(self, parameter_columns, headers, wb, sheet_name):
        selection_window = customtkinter.CTkToplevel(self.root)
        selection_window.title(f"Select Headers from Current Year Data for {sheet_name}")
        selection_window.geometry(f"500x600")
        selection_window.lift()  # Bring window to the front
        selection_window.attributes('-topmost', True)  # Keep it on top
        selection_window.focus_force()  # Force focus on the new window

        frame = customtkinter.CTkFrame(selection_window)
        frame.pack(fill="both", expand=True)

        selected_headers = {}

        for param in parameter_columns:
            var = StringVar(selection_window)
            # Auto-select the closest match if available
            best_match = self.get_best_match(param, headers)
            var.set(best_match)  # Set the best match found, otherwise remains "None"
            dropdown = customtkinter.CTkOptionMenu(frame, variable=var, values=headers, font=("Arial", 16))
            dropdown.configure(width=35)
            customtkinter.CTkLabel(frame, text=param, font=("Arial", 16)).grid(row=len(selected_headers), column=0, padx=5, pady=5)
            dropdown.grid(row=len(selected_headers), column=1, padx=5, pady=5)
            selected_headers[param] = var

        additional_params = ["Manually exclude (1 or exclude)", "Exclusion rationale", "Net Net Spends", "Quantity"]

        for param in additional_params:
            var = StringVar(selection_window)
            best_match = self.get_best_match(param, headers)
            var.set(best_match)
            dropdown = customtkinter.CTkOptionMenu(frame, variable=var, values=headers, font=("Arial", 16))
            dropdown.configure(width=35)
            customtkinter.CTkLabel(frame, text=param, font=("Arial", 16)).grid(row=len(selected_headers), column=0, padx=5, pady=5)
            dropdown.grid(row=len(selected_headers), column=1, padx=5, pady=5)
            selected_headers[param] = var

        def on_confirm():
            for param, var in selected_headers.items():
                self.selected_headers[param] = var.get()
                print(f"User selected header for {param}: {var.get()}")

            selection_window.destroy()
            self.copy_to_current_year_analysis(wb, sheet_name)

        customtkinter.CTkButton(frame, text="Confirm", command=on_confirm, font=("Arial", 16)).grid(row=len(selected_headers) + 1, column=0, columnspan=2, pady=10)

    def copy_to_current_year_analysis(self, wb, sheet_name):
        print(f"Copying data to 'Current Year Analysis' tab for {sheet_name}.")

        current_year_ws = wb.sheets['Current Year Analysis']
        last_row = current_year_ws.range('A' + str(current_year_ws.cells.last_cell.row)).end('up').row
        if last_row > 7:
            clear_range = f"A8:AZ{last_row}"
            current_year_ws.range(clear_range).clear_contents()

        additional_header_to_col = {"Manually exclude (1 or exclude)": 'S', "Exclusion rationale": 'T', "Net Net Spends": 'W', "Quantity": 'X'}
        start_col = 'A'

        for idx, (param, header) in enumerate(self.selected_headers.items()):
            if header == "None":
                print(f"No header selected for {param}. Skipping.")
                continue

            if header not in self.schedule_df.columns:
                print(f"Header '{header}' not found in the DataFrame. Skipping.")
                continue

            header_col = self.schedule_df.columns.get_loc(header)
            data_to_copy = self.schedule_df.iloc[:, header_col].values.reshape(-1, 1)

            if param in additional_header_to_col:
                target_col = additional_header_to_col[param]
                current_year_ws.range(f"{target_col}8").value = data_to_copy
            else:
                current_year_ws.range(f"{start_col}8").offset(0, idx).value = data_to_copy

        # Copying MS Unique ID data into AZ column
        if 'MS Unique ID' in self.schedule_df.columns:
            ms_unique_id_col = self.schedule_df.columns.get_loc('MS Unique ID')
            ms_unique_id_data = self.schedule_df.iloc[:, ms_unique_id_col].values.reshape(-1, 1)
            current_year_ws.range('AZ8').value = ms_unique_id_data
            print(f"Copied MS Unique ID data to AZ column in 'Current Year Analysis' tab.")
        else:
            print(f"'MS Unique ID' column not found in the uploaded data.")

        print(f"Data copied to 'Current Year Analysis' tab for {sheet_name}.")
        self.save_workbook(sheet_name)

    def save_workbook(self, sheet_name):
        new_filename = os.path.join(self.output_dir, f"Abacus_6.5_{sheet_name}.xlsm")
        self.wb.save(new_filename)
        print(f"Workbook saved as {new_filename}")

        self.selected_headers.clear()
        self.schedule_df = None
        self.current_sheet_index += 1
        self.process_next_sheet()

    def finish_processing(self):
        if not self.wb.app is None:
            self.wb.close()
            self.app.quit()
        messagebox.showinfo("Success", "Processing completed successfully.")
        print("Processing completed successfully.")

# Initialize customtkinter and run the app
root = customtkinter.CTk()
app = App(root)
root.mainloop()
